In [1]:
import os
dataset_name = "drkg-hsdn-custom-subset"
datadir = os.path.join(os.getcwd(), dataset_name, 'data')

In [8]:
import pandas as pd
df = pd.read_csv(os.path.join(datadir, 'raw_relations.tsv'), sep='\t')
df.head()

,relation_type,source_type,source_id,target_type,target_id,resource
0,bioarx::HumGenHumGen:Gene:Gene,Gene,ENTREZ:2157,Gene,ENTREZ:2157,bioarx
1,bioarx::HumGenHumGen:Gene:Gene,Gene,ENTREZ:2157,Gene,ENTREZ:5264,bioarx
2,bioarx::HumGenHumGen:Gene:Gene,Gene,ENTREZ:2157,Gene,ENTREZ:2158,bioarx
3,bioarx::HumGenHumGen:Gene:Gene,Gene,ENTREZ:2157,Gene,ENTREZ:3309,bioarx
4,bioarx::HumGenHumGen:Gene:Gene,Gene,ENTREZ:2157,Gene,ENTREZ:28912,bioarx


In [9]:
import networkx as nx

G = nx.Graph()  # Create an undirected graph

for index, row in df.iterrows():
    source_id = row['source_id']
    source_name = row.get('source_name', '')
    source_type = row['source_type']
    target_id = row['target_id']
    target_type = row['target_type']
    target_name = row.get('target_name', '')
    relation_type = row['relation_type']

    # Add source and target nodes to the graph
    G.add_node((source_id, source_type), name=source_name)
    G.add_node((target_id, target_type), name=target_name)

    # Add an edge between source and target nodes with relation type
    G.add_edge((source_id, source_type), (target_id, target_type), relation=relation_type)


In [10]:
# specified_node = ('specified_id', 'specified_type')
specified_node = ('MESH:D015673', 'Disease')
subgraph = G.subgraph(nx.node_connected_component(G, specified_node))


subgraph_nodes = list(subgraph.nodes())
subgraph_edges = list(subgraph.edges(data=True))

subgraph_data = []

# Iterate over the edges in the subgraph and extract relevant information
for edge in subgraph_edges:
    source_node, target_node, data = edge
    source_id = source_node[0]
    source_type = source_node[1]
    source_name = subgraph.nodes[source_node]['name']
    target_type = target_node[1]
    target_id = target_node[0]
    target_name = subgraph.nodes[target_node]['name']   
    relation_type = data['relation']
    
    # Append the information to the subgraph_data list
    subgraph_data.append({
        'source_id': source_id,
        'source_name': source_name,
        'source_type': source_type,
        'target_id': target_id,
        'target_name': target_name,
        'target_type': target_type,
        'relation_type': relation_type
    })

# Create a DataFrame from the subgraph data
subgraph_df = pd.DataFrame(subgraph_data)
outputfile = os.path.join(datadir, 'relations.tsv')
subgraph_df.to_csv(outputfile, index=False, sep='\t')

In [11]:
print(df.shape, subgraph_df.shape)

(6015458, 6) (4538859, 7)
